In [1]:
import numpy as np
import torch
import pandas as pd
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [227]:
class CustomDataset(Dataset):
    def __init__(self, filename, analyze_ticks, result_ticks, start_ticks, end_ticks, viborka):
        self.data_path = filename
        self.data = []
        df = pd.read_csv(self.data_path, encoding='utf-16', delimiter=";", header=None)
        ticksArray = np.array(df.values)
        if end_ticks>ticksArray.shape[0]:
            end_ticks=ticksArray.shape[0]
        if start_ticks>ticksArray.shape[0]:
            start_ticks=0
        # max_sum = 0;
        
        for idx in np.arange(analyze_ticks-1+start_ticks, end_ticks-result_ticks-1):
            
            analyze_set = np.array([])
            for i in np.arange(analyze_ticks-1, 0, -1):
                analyze_set = np.append(analyze_set, ticksArray[idx-i], 0)
            result_sum = 0
            for i in np.arange(result_ticks):
                result_sum+=ticksArray[idx+i+1][0]
            
            if(abs(result_sum)>viborka):
                if result_sum>0:
                    result_sum=[0,0,1]
                else:
                    result_sum=[1,0,0]
            else:
                result_sum = [0,1,0]
            
            # if abs(result_sum)>max_sum:
            #     max_sum=abs(result_sum)
            
            # if(abs(result_sum)>1000):
            #     self.data.append([analyze_set, result_sum])
            # elif np.random.rand()<viborka:
            #     self.data.append([analyze_set, result_sum])
            self.data.append([analyze_set, result_sum])
            
        # print(max_sum)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        analyze_set, result_sum = self.data[idx]
        analyze_tensor = torch.from_numpy(analyze_set).to(torch.float32)
        result_tensor = torch.tensor(result_sum).to(torch.float32)
        return analyze_tensor, result_tensor

In [29]:
dataset_train = CustomDataset(241,20,0,8000, 1000)
dataset_test = CustomDataset(241,20,8000, 10000, 1000)

batch_size0 = 100

train_dataloader = DataLoader(dataset_train, batch_size=batch_size0, shuffle=True)
test_dataloader = DataLoader(dataset_test, batch_size=batch_size0, shuffle=True)


In [213]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(240, 20),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(10, 3),
        )
        # print(self.linear_relu_stack[0].weight.dtype)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [214]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=240, out_features=20, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=20, out_features=10, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=10, out_features=3, bias=True)
  )
)


In [215]:
learning_rate = 1e-3


In [236]:
def train_loop(dataloader, model, loss_fn, optimizer, log):
    size = len(dataloader.dataset)
    one_time = True
    for batch, (X, y) in enumerate(dataloader):
        
        # Compute prediction and loss
        pred = model(X)
        
        if one_time:
            one_time= False
            # print(pred)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 30 == 0 and log:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, log):
    # size = len(dataloader.dataset)
    size = 0
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            for idx in np.arange(y.shape[0]):
                if(pred[idx].argmax()!=1):
                    size+=1
                    # if(pred[idx].argmax() == y[idx].argmax()):
                    if(y[idx].argmax()!=1):
                        correct += 1

    test_loss /= num_batches
    correct /= size
    if log:
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [217]:
# loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [224]:


epochs = 1
for t in range(epochs):
    log = False
    if (t+1)%1==0:
        log = True
        print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, log)
    test_loop(test_dataloader, model, loss_fn, log)
print("Done!")

Epoch 1
-------------------------------
loss: 0.677020  [  100/ 7739]
loss: 0.803435  [ 3100/ 7739]
loss: 0.761472  [ 6100/ 7739]
Test Error: 
 Accuracy: 57.2%, Avg loss: 1.324360 

Done!


In [226]:
test_loop(test_dataloader, model, loss_fn, True)

Test Error: 
 Accuracy: 57.2%, Avg loss: 1.368079 



In [237]:
for i in range(10):
    dataset_test = CustomDataset("testingData 20000.csv", 241, 20, i*2000, (i+1)*2000, 1000)

    batch_size0 = 100

    test_dataloader = DataLoader(dataset_test, batch_size=batch_size0, shuffle=True)
    test_loop(test_dataloader, model, loss_fn, True)

Test Error: 
 Accuracy: 83.3%, Avg loss: 1.302250 

Test Error: 
 Accuracy: 82.0%, Avg loss: 1.319752 

Test Error: 
 Accuracy: 76.2%, Avg loss: 1.214361 

Test Error: 
 Accuracy: 75.2%, Avg loss: 1.201470 

Test Error: 
 Accuracy: 72.9%, Avg loss: 1.150988 

Test Error: 
 Accuracy: 81.3%, Avg loss: 0.784811 

Test Error: 
 Accuracy: 87.8%, Avg loss: 0.676297 

Test Error: 
 Accuracy: 91.2%, Avg loss: 0.682871 

Test Error: 
 Accuracy: 90.5%, Avg loss: 0.632325 

Test Error: 
 Accuracy: 78.3%, Avg loss: 1.365291 



Test Error: 
 Accuracy: 72.8%, Avg loss: 0.828556 



In [225]:
torch.save(model, "model 241 20 v1 H1 10000 win57.pth")

In [ ]:
model = torch.load("model 51 20 v3 perc6.pth")

In [14]:
epochs=10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    test_loop(test_dataloader, model, loss_fn, 3, True)

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2344.025424 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.977581 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2342.015881 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.980488 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2342.068672 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.973897 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.983285 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2342.107387 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.960891 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 2.0%, Avg loss: 2341.968734 

